In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
from keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from gymnasium.wrappers import FrameStack
from gymnasium.experimental.wrappers import GrayscaleObservationV0
import gymnasium as gym
import numpy as np
import tensorflow as tf
import pandas as pd
from IPython.display import clear_output
%config NotebookApp.iopub_msg_rate_limit=10000
import time

In [2]:
num_actions = 5
def create_CNN():
    model = Sequential()
    
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='same', activation='relu', input_shape=(4, 210, 160)))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu'))
    
    model.add(Flatten())
    
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_actions, activation='linear'))
    return model

In [3]:
model = create_CNN()
model.load_weights('cnn1v2.weights.h5')
env = gym.make('ALE/Frogger-v5')
env = GrayscaleObservationV0(env)
env = FrameStack(env, 4) # Get 4 frames from game at a time
frames, width, height = env.observation_space.shape
env.reset()
game_over = False

C:\Users\zmajernik\AppData\Roaming\Python\Python39\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [10]:
def testAI(episodes):
    stepsAndScore = {'steps': [], 'score': []}
    for i in range(episodes):
        totalSteps = 0
        totalScore = 0
        env.reset()
        state, reward, game_over, x, _ = env.step(0)
        while not game_over:
            #Predict action using the trained model
            q_values = model.predict(np.expand_dims(state, axis=0))
            action = np.argmax(q_values)

            # Take action in the environment
            next_state, reward, game_over, x, _ = env.step(action)

            totalScore += reward
            totalSteps += 1

            # Update current state
            state = next_state
            
            time.sleep(0.001)
            clear_output(wait=True)
            print("episode " + str(i + 1) + "/" + str(episodes))
        stepsAndScore['steps'].append(totalSteps)
        stepsAndScore['score'].append(totalScore)
    stepsAndScore = pd.DataFrame(stepsAndScore)
    return stepsAndScore

In [19]:
#set number of episodes to test the AI for
episodesToTest = 20

In [20]:
stepsAndScore = testAI(episodesToTest)

episode 5/5


In [21]:
stepsAndScore

,steps,score
0,287,9.0
1,338,11.0
2,338,11.0
3,339,11.0
4,338,11.0


In [26]:
bestScore = max(stepsAndScore['score'])
avgScore = stepsAndScore['score'].mean()
avgStepsPerScore = round(stepsAndScore['steps'].mean()/avgScore,2)

In [28]:
print("Best Score: " + str(bestScore))
print("Average Score: " + str(avgScore))
print("Average Steps/Score: " + str(avgStepsPerScore))

Best Score: 11.0
Average Score: 10.6
Average Steps/Score: 30.94
